In [1]:
import torch, json
from math import floor
import pandas as pd
import numpy as np

from FileManager.dataManager import dataManager
from AnalyzeTools.models import autoregressive_integrated_moving_average, linear_regression, support_vector_regression, random_forest, gradient_boosting
from AnalyzeTools.prepare import data_split, model_eval, pathForSavingModels
from AnalyzeTools.preprocess import preprocessData, removeOutliers
from AnalyzeTools.superModels import DEEPAR, TFT, RNN

c:\Users\zyf13\AppData\Local\Programs\Python\Python37\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Global seed set to 123


In [2]:
period = 'Day'
future_step = 1
params_path = f'./Models'

product_object = json.load(open("./File information.json", "r", encoding='utf8'))

all_experiments= []
for product in product_object.keys():
    for raw_file_name in  product_object[product].keys():
        for product_type in product_object[product][raw_file_name]['product_types']:
            for target in product_object[product][raw_file_name]['targets']:
                all_experiments.append([product, raw_file_name, product_type, target])

In [3]:
n = 0
experiment = all_experiments[n]
product, raw_file_name, product_type, target = experiment
print(f"Product: {product}\nRaw file name: {raw_file_name}\nProduct_type: {product_type}\ntarget: {target}")

Product: pork
Raw file name: (중)경략가격집계 - 소,돼지
Product_type: 돼지 온도체
target: MAX_COST_AMT


In [4]:
df, product_and_product_type, product_attribute = dataManager(raw_file_name, product, product_type, target)

In [10]:
test_data = df.copy()

test_data['date'] = pd.to_datetime(test_data['date'])
test_data['year'] = test_data['date'].dt.year
test_data['month'] = test_data['date'].dt.month
test_data['week'] = test_data['date'].dt.isocalendar().week
test_data['day'] = test_data['date'].dt.day

test_data = test_data[['date', 'year', 'month', 'week', 'day', 'JUDGE_KIND', 'CNT', 'MIN_COST_AMT', 'MAX_COST_AMT',
                        'SUM_COST_AMT', 'SUM_WEIGHT', 'STATUS', 'DEFECT_CNT',
                        'DEFECT_MIN_COST_AMT', 'DEFECT_MAX_COST_AMT', 'DEFECT_SUM_COST_AMT',
                        'DEFECT_SUM_WEIGHT', 'DIEOUT_CNT', 'DIEOUT_MIN_COST_AMT',
                        'DIEOUT_MAX_COST_AMT', 'DIEOUT_SUM_COST_AMT', 'DIEOUT_SUM_WEIGHT',
                        'ETC']]

In [18]:
group_cols = ['year', 'month']
operations = ['first', None, None, 'first', 'first', 
 'mean', 'mean', 'mean', 'mean', 'mean', 
 'mean', 'mean', 'mean', 'mean', 'mean', 
 'mean', 'mean', 'mean', 'mean', 'mean', 
 'mean', 'mean', 'mean']

test_data.groupby(group_cols).agg({col: operation for col, operation in zip(test_data.columns, operations) if operation})

date  week  day  JUDGE_KIND        CNT  MIN_COST_AMT  \
year week                                                              
2012 1    2012-01-02     1    2         2.0  24.384987   4088.771175   
     2    2012-01-09     2    9         2.0  32.379860   3813.932000   
     3    2012-01-16     3   16         2.0  28.263979   3421.126318   
     4    2012-01-25     4   25         2.0  24.635609   3327.836650   
     5    2012-01-30     5   30         2.0  22.904857   3522.391994   
...              ...   ...  ...         ...        ...           ...   
2022 28   2022-07-11    28   11         2.0  43.989057   4266.821601   
     29   2022-07-18    29   18         2.0  47.879120   4128.672043   
     30   2022-07-25    30   25         2.0  49.085532   3988.091638   
     31   2022-08-01    31    1         2.0  46.501590   4036.207587   
     32   2022-08-08    32    8         2.0  53.323391   4166.177782   

           MAX_COST_AMT  SUM_COST_AMT   SUM_WEIGHT  STATUS  ...  \
year week                                                   ...   
2012 1      5083.238704  9.779300e+06  2146.220037     NaN  ...   
     2      5038.560769  1.292347e+07  2768.893755     NaN  ...   
     3      4454.655889  1.010080e+07  2443.050137     NaN  ...   
     4      4207.336911  8.704762e+06  2162.708784     NaN  ...   
     5      4292.515900  8.068996e+06  2042.149377     NaN  ...   
...                 ...           ...          ...     ...  ...   
2022 28     5985.633335  2.178932e+07  3984.560648     NaN  ...   
     29     5977.091980  2.327646e+07  4350.192106     NaN  ...   
     30     5981.745409  2.353204e+07  4426.194009     NaN  ...   
     31     6080.744709  2.237492e+07  4155.663156     NaN  ...   
     32     6736.169497  2.814386e+07  4714.345070     NaN  ...   

           DEFECT_MIN_COST_AMT  DEFECT_MAX_COST_AMT  DEFECT_SUM_COST_AMT  \
year week                                                                  
2012 1                     NaN                  NaN                  NaN   
     2                     NaN                  NaN                  NaN   
     3                     NaN                  NaN                  NaN   
     4                     NaN                  NaN                  NaN   
     5                     NaN                  NaN                  NaN   
...                        ...                  ...                  ...   
2022 28            4266.821601          5985.633335         2.178932e+07   
     29            4128.672043          5977.091980         2.327646e+07   
     30            3988.091638          5981.745409         2.353204e+07   
     31            4036.207587          6080.744709         2.237492e+07   
     32            4166.177782          6736.169497         2.814386e+07   

           DEFECT_SUM_WEIGHT  DIEOUT_CNT  DIEOUT_MIN_COST_AMT  \
year week                                                       
2012 1                   NaN         NaN                  NaN   
     2                   NaN         NaN                  NaN   
     3                   NaN         NaN                  NaN   
     4                   NaN         NaN                  NaN   
     5                   NaN         NaN                  NaN   
...                      ...         ...                  ...   
2022 28          3984.560648         NaN                  NaN   
     29          4350.192106         NaN                  NaN   
     30          4426.194009         NaN                  NaN   
     31          4155.663156         NaN                  NaN   
     32          4714.345070         NaN                  NaN   

           DIEOUT_MAX_COST_AMT  DIEOUT_SUM_COST_AMT  DIEOUT_SUM_WEIGHT  ETC  
year week                                                                    
2012 1                     NaN                  NaN                NaN  NaN  
     2                     NaN                  NaN                NaN  NaN  
     3                     NaN                  NaN         

In [ ]:
if len(df) == 0:
    raise ValueError("No data!")

df, input_features = preprocessData(df, 'date', target, 'Day')
test_size = 0.2

df = removeOutliers(df, test_size, target)

train_x_axis = df['date'][:-1*floor(len(df) * test_size)].values if type(test_size) == float else df['date'][:-1*test_size]
predictions_x_axis = df['date'][-1*floor(len(df) * test_size):].values if type(test_size) == float else df['date'][-1*test_size:]

In [ ]:
# prepare dataset for statistics and Macnhine models
ml_split_params = {'Model': 'ML', 'Future': future_step}
X_train, X_test, y_train, y_test, input_scaler, output_scaler = data_split(df, input_features, target, test_size, scaling=True, **ml_split_params)

''' Input data into models and Evaluate model results '''
ml_searchCV_params = {
    'base_dir': params_path,
    'product_and_product_type': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'predict_type': 'single',
    'period': 'Day',
    'step': future_step,
    'save': True
}
stdout = True
vis = True

# Statistics

In [ ]:
# print("\nARIMA")
# arima_predictions = autoregressive_integrated_moving_average(y_train, y_test)
# model_eval(y_test, arima_predictions, stdout=stdout, vis=vis)

In [ ]:
print("\nLinear Regression")
lr, _ = linear_regression(X_train, y_train)
lr_predictions = lr.predict(X_test)
model_eval(y_test, lr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

# Machine learning

In [ ]:
print("\nSupport Vector Regression")
svr, _ = support_vector_regression(X_train, y_train, search=True, **ml_searchCV_params)
svr_predictions = svr.predict(X_test)
model_eval(y_test, svr_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

In [ ]:
print("\nRandom Forest")
rf, _ = random_forest(X_train, y_train, search=True, **ml_searchCV_params)
rf_predictions = rf.predict(X_test)
model_eval(y_test, rf_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

In [ ]:
print("\nGradient Boosting")
gb, _ = gradient_boosting(X_train, y_train, search=True, **ml_searchCV_params)
gb_predictions = gb.predict(X_test)
model_eval(y_test, gb_predictions, predictions_x_axis, stdout=stdout, vis=vis, **{'scaler': output_scaler})

# Deep learning

In [ ]:
data = df.copy()

# if future_step != 1:
#     dl_target = f'{target}_lead_{future_step}'
#     data[dl_target] = data[target].shift(-1 * (future_step - 1))
#     data = data[:-1 * (future_step - 1)]

data['time_idx'] = range(len(data))
data['group'] = product

training_cutoff = floor(len(data) * (1-test_size)) if type(test_size) == float else len(data) - test_size

max_prediction_length = 1 # In the case of point forecasting, this parameter is 1 by default
max_encoder_length = 30 # 7, 14, 30, 60, 120
batch_size = 64

group = ['group']
time_varying_known_categoricals = ['month', 'week']
time_varying_unknown_categoricals = []
time_varying_known_reals = ['time_idx']
time_varying_unknown_reals = input_features + [target]

# if future_step != 1:
#     time_varying_unknown_reals = input_features + [target, dl_target]
#     target_arg = dl_target
# else:
#     time_varying_unknown_reals = input_features + [target]
#     target_arg = target

In [ ]:
dl_searchCV_params = {
    'base_dir': params_path,
    'product_and_product_type': product_and_product_type,
    'attribute': product_attribute,
    'raw': raw_file_name,
    'predict_type': 'single',
    'period': 'Day',
    'step': future_step,
    'save': True
}

In [ ]:
print("\nLSTM")
training_params = {'max_epochs': 100, 'n_trials': 30}
saving_dir = pathForSavingModels('LSTM', **dl_searchCV_params)
lstm, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    saving_dir,
    'LSTM',
    training_params,
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
lstm_predictions = lstm.predict(val_dataloader)
model_eval(actuals, lstm_predictions, predictions_x_axis, stdout=True, vis=True)

In [ ]:
print("\nGRU")
training_params = {'max_epochs': 100, 'n_trials': 30}
saving_dir = pathForSavingModels('GRU', **dl_searchCV_params)
gru, val_dataloader = RNN(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    saving_dir,
    'GRU',
    training_params
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
gru_predictions = gru.predict(val_dataloader)

model_eval(actuals, gru_predictions, predictions_x_axis, stdout=True, vis=True)

In [ ]:
print("\nDeepAR")
training_params = {'max_epochs': 100, 'n_trials': 30}
saving_dir = pathForSavingModels('DeepAR', **dl_searchCV_params)
deep_ar, val_dataloader = DEEPAR(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_known_categoricals,
    time_varying_unknown_categoricals,
    time_varying_known_reals,
    batch_size,
    saving_dir,
    training_params
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
deepar_predictions = deep_ar.predict(val_dataloader)

model_eval(actuals, deepar_predictions, predictions_x_axis, stdout=True, vis=True)

# Transformer

In [ ]:
print("\nTFT")
training_params = {'max_epochs': 50, 'n_trials': 10}
saving_dir = pathForSavingModels('TFT', **dl_searchCV_params)
tft, val_dataloader = TFT(
    data,
    training_cutoff,
    target,
    group,
    max_encoder_length,
    max_prediction_length,
    time_varying_unknown_categoricals,
    time_varying_known_categoricals,
    time_varying_known_reals,
    time_varying_unknown_reals,
    batch_size,
    saving_dir,
    training_params
)

actuals = torch.cat([y[0] for x, y in iter(val_dataloader)])
tft_predictions = tft.predict(val_dataloader)

model_eval(actuals, tft_predictions, predictions_x_axis, stdout=True, vis=True)